In [4]:
using Pkg
using IJulia
using DifferentialEquations
using Plots, CSV, XLSX, DataFrames
using Peaks, Statistics,Roots, NaNStatistics, Random

function freeR(rt,at,ks)
    return (rt .- at .- ks .+ sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function freeA(rt,at,ks)
    return (at .- rt .- ks .+ sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function coeffI(rt,at,ka,ks,kb,kd)
    return (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)) ./ (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)*(ks*kd/(ka*kb)))
end
function coeffJ(rt,at,ka,ks,kb,kd)
    return (ks .+ ka .+ freeR(rt,at,ks)) ./ (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)*(ks*kd/(ka*kb)))
end
function tranBSD(rt,at,ka,ks,kb,kd)
    return (coeffI(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) ./ (1 .+ (coeffI(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) .+ (coeffJ(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) .* (freeR(rt,at,ks) ./ kb))
end
function modelDro!(dp,p,pa,t)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd) = pa
    dp[1] = tranBSD(p[3],AT,Ka,Ks,Kb,Kd)-b1*p[1]
    dp[2] = p[1]-b2*p[2]-a3*p[2]
    dp[3] = a3*p[2]-b3*p[3]
end
function ddmeasure(ts)
    ts = movmean(ts,100)
    peaks = findmaxima(ts)
    ths = findminima(ts)
    if min(length(peaks[1]),length(ths[1])) >= 5
        ind = min(length(peaks[1]),length(ths[1]))
        amps = abs.(peaks[2][1:ind] - ths[2][1:ind])
        periods = 0.01*diff(peaks[1])
        if (abs(amps[end]-amps[end-1])/amps[end] < 0.1) & (abs(periods[end]-periods[end-1])/periods[end] < 0.1)
            costf, amp, period, lev, relamp = max(0,5*(2-amps[end]/amps[end-1]-amps[end-1]/amps[end-2])), amps[end], periods[end], (peaks[2][end]+ths[2][end])/2, amps[end]/peaks[2][end]
        else
            costf, amp, period, lev, relamp = max(0,5*(2-amps[end]/amps[end-1]-amps[end-1]/amps[end-2])), 0, 0, ts[end], 0
        end
    else
        costf, amp, period, lev, relamp = 10, 0, 0, ts[end], 0
    end
    return costf, amp, period, lev, relamp
end
function ddsa(pa)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd) = pa
    timept = [0, 4, 8, 12, 16, 20]
    pcmMper = [54.59242, 57.97024, 39.78646, 37.68333, 38.33725, 47.00128]
    pcmMper = pcmMper/maximum(pcmMper)
    pcmPER = [1180.3421, 1028.4309, 1521.4779, 1305.6787, 895.0438, 821.742]
    pcmPER = pcmPER/maximum(pcmPER)
    prob = ODEProblem(modelDro!,zeros(3),(0.0,300.0),pa)
    ts = solve(prob,saveat=0.01,Rosenbrock23())
    mRNAs = ts[1,:]
    proteins = ts[3,:]
    cost = ddmeasure(mRNAs)[1]+
           ddmeasure(proteins)[1]+
           sqrt((1-ddmeasure(mRNAs)[5]/(1-minimum(pcmMper)))^2+     # mRNA relamp 
                (1-ddmeasure(proteins)[5]/(1-minimum(pcmPER)))^2+   # PER relamp
                (1-ddmeasure(proteins)[3]/24)^2)                    # PER period
    return cost
end
function ddsa_2(pa)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd) = pa
    timept = [0, 4, 8, 12, 16, 20]
    pcmMper = [54.59242, 57.97024, 39.78646, 37.68333, 38.33725, 47.00128]
    pcmMper = pcmMper/maximum(pcmMper)
    pcmPER = [1180.3421, 1028.4309, 1521.4779, 1305.6787, 895.0438, 821.742]
    pcmPER = pcmPER/maximum(pcmPER)

    prob = ODEProblem(modelDro!,zeros(3),(0.0,300.0),pa)
    ts = solve(prob,saveat=0.01,Rosenbrock23())
    mRNAs = ts[1,:]
    proteins = ts[3,:]

    shift_mRNA = 0
    shift_PER = 0
    
    mRNA_peak_value_index_0_to_24 = argmax(ts[1, 20900:23300])/100           # 먼저 shift하지 않은 상태에서 peak값을 갖는 index (0-24)를 구한다. 20900을 index 1로 잡는다. 
    shift_mRNA = Int(trunc((mRNA_peak_value_index_0_to_24-4)*100))           # shift하지 않은 상태에서 peak값을 갖는 index (0-24)값과, 실제 peak index값인 4를 비교를 해서, shift값을 계산한다. 
    scale_mRNA = maximum(ts[1,20900+shift_mRNA:23300+shift_mRNA])            # peak 값이 겹치도록 계산한 mRNA time series를 평행이동한다. 
    
    mRNA_shifted_peak_value_index_0_to_24 = argmax(ts[1,20900+shift_mRNA:23300+shift_mRNA])/100       # shift한 mRNA에서 peak값을 갖는 시간값/index (0-24) 를 계산한다. 
    mRNA_shifted_trough_value_index_0_to_24 = argmin(ts[1,20900+shift_mRNA:23300+shift_mRNA])/100     # shift한 mRNA에서 trough값을 갖는 시간값/index (0-24) 를 계산한다. 
    mRNA_shifted_trough_value = minimum(ts[1,20900+shift_mRNA:23300+shift_mRNA]/scale_mRNA)           # shift한 mRNA에서 trough값을 계산한다. 
    
    PER_peak_value_index_0_to_24 = argmax(ts[3, 20000:22400])/100                 # 먼저 shift하지 않은 상태에서 peak값을 갖는 index (0-24)를 구한다. 20900을 index 1로 잡는다.  
    shift_PER = Int(trunc((PER_peak_value_index_0_to_24-8)*100))                  # shift하지 않은 상태에서 peak값을 갖는 index (0-24)값과, 실제 peak index값인 4를 비교를 해서, shift값을 계산한다. 
    scale_PER = maximum(ts[3,20000+shift_PER:22400+shift_PER])
    
    PER_shifted_peak_value_index_0_to_24 = argmax(ts[3,20000+shift_PER:22400+shift_PER])/100          # shift한 PER에서 peak 값을 갖는 시간값/index (0-24) 를 계산한다. 
    PER_shifted_trough_value_index_0_to_24 = argmin(ts[3,20000+shift_PER:22400+shift_PER])/100        # shift한 PER에서 trough 값을 갖는 시간값/index (0-24) 를 계산한다. 
    PER_shifted_trough_value = minimum(ts[3,20000+shift_PER:22400+shift_PER]/scale_PER)               # shift한 PER에서 trough 값을 계산한다. 
    
    cost_mRNA_trough_time = (1 - mRNA_shifted_trough_value_index_0_to_24/12)^2
    cost_mRNA_trough_value = (1 - mRNA_shifted_trough_value/0.65)^2
    
    cost_PER_trough_time = (1 - PER_shifted_trough_value_index_0_to_24/20)^2
    cost_PER_trough_value = (1 - PER_shifted_trough_value/0.54)^2
    
    cost_mRNA_minT_maxT = (1-abs(mRNA_shifted_trough_value_index_0_to_24 - 4)/12)^2

    cost = ddmeasure(mRNAs)[1]+
           ddmeasure(proteins)[1]+
           sqrt((1-ddmeasure(mRNAs)[5]/(1-minimum(pcmMper)))^2+     # mRNA relamp 
                2.5*(1-ddmeasure(proteins)[5]/(1-minimum(pcmPER)))^2+   # PER relamp
                1.5*(1-ddmeasure(mRNAs)[3]/24)^2+                   # PER period
                1.5*(1-ddmeasure(proteins)[3]/24)^2+                # PER period
                0.6*cost_mRNA_minT_maxT)                        # mRNA minT - maxT 시간 차이 맞추는 cost
                                                           
    return cost
end

function modelWT!(dp,p,pa,t)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd,ampD,phD,ampT,phT) = pa
    dp[1] = tranBSD(p[3],AT,Ka,Ks,Kb,Kd)-rhythm(t,ampD,24,phD,b1)p[1]
    dp[2] = rhythm(t,ampT,24,phT,1)*p[1]-b2*p[2]-a3*p[2]
    dp[3] = a3*p[2]-b3*p[3]
end
# Define the simulated annealing function
function simulated_annealing(objective_function, initial_state, max_iterations, initial_temp, alpha)
    # Initialize the state
    current_state = initial_state
    current_value = objective_function(current_state)
    best_state = current_state
    best_value = current_value
    # Initialize temperature
    temp = initial_temp
    for i in 1:max_iterations
        # Generate a new candidate state
        candidate_state = perturb(current_state)
        candidate_value = objective_function(candidate_state)
        # Calculate the change in the objective function value
        delta_value = candidate_value - current_value
        # Decide whether to accept the new candidate state
        if delta_value < 0 || exp(-delta_value / temp) > rand()
            current_state = candidate_state
            current_value = candidate_value
        end
        # Update the best state found so far
        if current_value < best_value
            best_state = current_state
            best_value = current_value
        end
        # Decrease the temperature
        temp *= alpha
        # Print progress
        if i%100==0
            println("Iteration $i: Best Para = $best_state, Best value = $best_value, Temperature = $temp")
        end
    end
    return best_state, best_value
end
function perturb(pa)
    newpa = (pa[1]*exp(0.2*rand(1)[1]-0.1),pa[2]*exp(0.2*rand(1)[1]-0.1),pa[3]*exp(0.2*rand(1)[1]-0.1),pa[4]*exp(0.2*rand(1)[1]-0.1),pa[5]*exp(0.2*rand(1)[1]-0.1),pa[6]*exp(0.2*rand(1)[1]-0.1),pa[7]*exp(0.2*rand(1)[1]-0.1),pa[8]*exp(0.2*rand(1)[1]-0.1),pa[9]*exp(0.2*rand(1)[1]-0.1))
    return newpa
end


perturb (generic function with 1 method)

In [5]:
# Parameters order: a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd
costf = Inf
iteration_count = 0

while costf > 1
    global paInit=(10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7),10.0^(7*rand(1)[1]-7))
    # costf = ddsa_2(paInit) 
    prob = ODEProblem(modelDro!,zeros(3),(0.0,300.0),paInit)
    detDro = solve(prob,saveat=0.01,Rosenbrock23())
    costf = ddmeasure(detDro[1,:])[1]+ddmeasure(detDro[3,:])[1]
    
    iteration_count += 1

    if iteration_count % 100 == 0
        println("Progress: $iteration_count iterations")
    end    
end
println(costf)
println(paInit)

best_state, best_value = simulated_annealing(ddsa_2, paInit, 40000, 1, 0.9995)

best_state, best_value


Progress: 100 iterations
Progress: 200 iterations
Progress: 300 iterations
Progress: 400 iterations
Progress: 500 iterations
Progress: 600 iterations
Progress: 700 iterations
Progress: 800 iterations
Progress: 900 iterations
Progress: 1000 iterations
Progress: 1100 iterations
Progress: 1200 iterations
0.002394275675028368
(0.19468207964554515, 0.05896050892930234, 0.09856865971027425, 0.25876520138152076, 0.45279876849041845, 3.4270111793035147e-6, 9.993959258444356e-7, 4.2640881439932686e-7, 2.0752243894431293e-5)
Iteration 100: Best Para = (0.18504966617218951, 0.063252085137018, 0.096688669996933, 0.437426256725854, 0.4924233013320858, 3.205969417971593e-6, 1.6090100436947194e-6, 3.606993917552564e-7, 1.884734612569884e-5), Best value = 0.9305733535914538, Temperature = 0.951217530242334
Iteration 200: Best Para = (0.18504966617218951, 0.063252085137018, 0.096688669996933, 0.437426256725854, 0.4924233013320858, 3.205969417971593e-6, 1.6090100436947194e-6, 3.606993917552564e-7, 1.884

((1.1604006953791408, 0.03429230676284004, 4.101027348471566e-6, 1.0314143639769924, 0.040398923653623416, 7.696067145153723e-11, 6.084086532311068e-8, 1.1582950393742907e-9, 0.0010675952157279862), 0.6941204169200573)

In [ ]:
(1.1604006953791408, 0.03429230676284004, 4.101027348471566e-6, 1.0314143639769924, 0.040398923653623416, 7.696067145153723e-11, 6.084086532311068e-8, 1.1582950393742907e-9, 0.0010675952157279862), 0.6941204169200573)